# wordslab-notebooks-lib.tools

> Predefined tools for LLMs

In [4]:
#| default_exp tools

In [5]:
#| export
from bs4 import BeautifulSoup
from cloudscraper import create_scraper
from html2text import HTML2Text
from pathlib import Path
from urllib.parse import urljoin, urlparse
import re
from textwrap import dedent

As of January 2026, Anthropic models and Claude Code are the reference for AI coding agents.

We will implement in priority all the tools described in: https://platform.claude.com/docs/en/agents-and-tools/tool-use/overview

## Text editor tool

The model can use an the text editor tool to view and modify text files, helping you debug, fix, and improve your code or other text documents. This allows the model to directly interact with your files, providing hands-on assistance rather than just suggesting changes.

**When to use the text editor tool**

Some examples of when to use the text editor tool are:
- Code debugging: Have the model identify and fix bugs in your code, from syntax errors to logic issues.
- Code refactoring: Let the model improve your code structure, readability, and performance through targeted edits.
- Documentation generation: Ask the model to add docstrings, comments, or README files to your codebase.
- Test creation: Have the model create unit tests for your code based on its understanding of the implementation.

**Use the text editor tool**

Provide the text editor tool (named `str_replace_based_edit_tool`) to the model. 

You can optionally specify a `max_characters` parameter to control truncation when viewing large files.

The text editor tool can be used in the following way:

1 Provide the model with the text editor tool and a user prompt

- Include the text editor tool in your API call
- Provide a user prompt that may require examining or modifying files, such as "Can you fix the syntax error in my code?"

2 The model uses the tool to examine files or directories

- The model assesses what it needs to look at and uses the view command to examine file contents or list directory contents
- The model response will contain a tool use request with the view command

3 Execute the view command and return results

- Extract the file or directory path from the model's tool use request
- Read the file's contents or list the directory contents
- If a max_characters parameter was specified in the tool configuration, truncate the file contents to that length
- Return the results to the model by continuing the conversation with a new user message containing a tool result

4 The model uses the tool to modify files

- After examining the file or directory, the model may use a command such as str_replace to make changes or insert to add text at a specific line number.
- If the model uses the str_replace command, it constructs a properly formatted tool use request with the old text and new text to replace it with

5 Execute the edit and return results

- Extract the file path, old text, and new text from the model's tool use request
- Perform the text replacement in the file
- Return the results to the model

6 The model provides its analysis and explanation

- After examining and possibly editing the files, the model provides a complete explanation of what it found and what changes it made

**Text editor tool commands**

The text editor tool supports several commands for viewing and modifying files:

**view**

The `view` command allows the model to examine the contents of a file or list the contents of a directory. It can read the entire file or a specific range of lines.

Parameters:
- command: Must be "view"
- path: The path to the file or directory to view
- view_range (optional): An array of two integers specifying the start and end line numbers to view. Line numbers are 1-indexed, and -1 for the end line means read to the end of the file. This parameter only applies when viewing files, not directories.

Example view commands

```json
// Example for viewing a file
{
  "input": {
    "command": "view",
    "path": "primes.py"
  }
}

// Example for viewing a directory
{
  "input": {
    "command": "view",
    "path": "src/"
  }
}
```

**str_replace**

The `str_replace` command allows the model to replace a specific string in a file with a new string. This is used for making precise edits.

Parameters:
- command: Must be "str_replace"
- path: The path to the file to modify
- old_str: The text to replace (must match exactly, including whitespace and indentation)
- new_str: The new text to insert in place of the old text

Example str_replace command

```json
{
  "input": {
    "command": "str_replace",
    "path": "primes.py",
    "old_str": "for num in range(2, limit + 1)",
    "new_str": "for num in range(2, limit + 1):"
  }
}
```

**create**

The `create` command allows the model to create a new file with specified content.

Parameters:
- command: Must be "create"
- path: The path where the new file should be created
- file_text: The content to write to the new file

Example create command

```json
{
  "input": {
    "command": "create",
    "path": "test_primes.py",
    "file_text": "import unittest\nimport primes\n\nclass TestPrimes(unittest.TestCase):\n    def test_is_prime(self):\n        self.assertTrue(primes.is_prime(2))\n        self.assertTrue(primes.is_prime(3))\n        self.assertFalse(primes.is_prime(4))\n\nif __name__ == '__main__':\n    unittest.main()"
  }
}
```

**insert**

The `insert` command allows the model to insert text at a specific location in a file.

Parameters:
- command: Must be "insert"
- path: The path to the file to modify
- insert_line: The line number after which to insert the text (0 for beginning of file)
- new_str: The text to insert

Example insert command

```json
{
  "input": {
    "command": "insert",
    "path": "primes.py",
    "insert_line": 0,
    "new_str": "\"\"\"Module for working with prime numbers.\n\nThis module provides functions to check if a number is prime\nand to generate a list of prime numbers up to a given limit.\n\"\"\"\n"
  }
}
```

**Implement the text editor tool**

1 Initialize your editor implementation

Create helper functions to handle file operations like reading, writing, and modifying files. Consider implementing backup functionality to recover from mistakes.

2 Handle editor tool calls

Create a function that processes tool calls from the model based on the command type:

3 Implement security measures

Add validation and security checks:
- Validate file paths to prevent directory traversal
- Create backups before making changes
- Handle errors gracefully
- Implement permissions checks

When implementing the text editor tool, keep in mind:
- Security: The tool has access to your local filesystem, so implement proper security measures.
- Backup: Always create backups before allowing edits to important files.
- Validation: Validate all inputs to prevent unintended changes.
- Unique matching: Make sure replacements match exactly one location to avoid unintended edits.

**Handle errors**

File not found

If the model tries to view or modify a file that doesn't exist, return an appropriate error message in the tool_result: "Error: File not found"

Multiple matches for replacement

If the str_replace command matches multiple locations in the file, return an appropriate error message: "Error: Found 3 matches for replacement text. Please provide more context to make a unique match."

No matches for replacement

If the str_replace command doesn't match any text in the file, return an appropriate error message: "Error: No match found for replacement. Please check your text and try again."

Permission errors

If there are permission issues with creating, reading, or modifying files, return an appropriate error message: "Error: Permission denied. Cannot write to file."

**Implementation best practices**

Provide clear context

When asking the model to fix or modify code, be specific about what files need to be examined or what issues need to be addressed. Clear context helps the model identify the right files and make appropriate changes.

- Less helpful prompt: "Can you fix my code?"
- Better prompt: "There's a syntax error in my primes.py file that prevents it from running. Can you fix it?"

Be explicit about file paths

Specify file paths clearly when needed, especially if you're working with multiple files or files in different directories.
- Less helpful prompt: "Review my helper file"
- Better prompt: "Can you check my utils/helpers.py file for any performance issues?"

Create backups before editing

Implement a backup system in your application that creates copies of files before allowing the model to edit them, especially for important or production code.

Handle unique text replacement carefully

The str_replace command requires an exact match for the text to be replaced. Your application should ensure that there is exactly one match for the old text or provide appropriate error messages.

```python
if count == 0:
    return "Error: No match found"
elif count > 1:
    return f"Error: Found {count} matches"
else:
    ...
    return "Successfully replaced text"
```

Verify changes

After the model makes changes to a file, verify the changes by running tests or checking that the code still works as expected.

**Implementation from https://github.com/AnswerDotAI/claudette**

Implements functions for Anthropic's Text Editor Tool API, allowing a model to view and edit files.

In [16]:
#| exports
def view(path:str,  # The path to the file or directory to view
         view_range:tuple[int,int]=None, # Optional array of two integers specifying the start and end line numbers to view. Line numbers are 1-indexed, and -1 for the end line means read to the end of the file. This parameter only applies when viewing files, not directories.
         nums:bool=False # Optionally prefix all lines of the file with a line number
        ) -> str:
    'Examine the contents of a file or list the contents of a directory. It can read the entire file or a specific range of lines. With or without line numbers.'
    try:
        p = Path(path).expanduser().resolve()
        if not p.exists(): return f'Error: File not found: {p}'
        if p.is_dir():
            res = [str(f) for f in p.glob('**/*') 
                   if not any(part.startswith('.') for part in f.relative_to(p).parts)]
            return f'Directory contents of {p}:\n' + '\n'.join(res)
        
        lines = p.read_text().splitlines()
        s,e = 1,len(lines)
        if view_range:
            s,e = view_range
            if not (1 <= s <= len(lines)): return f'Error: Invalid start line {s}'
            if e != -1 and not (s <= e <= len(lines)): return f'Error: Invalid end line {e}'
            lines = lines[s-1:None if e==-1 else e]
            
        return '\n'.join([f'{i+s-1:6d} │ {l}' for i,l in enumerate(lines,1)] if nums else lines)
    except Exception as e: return f'Error viewing file: {str(e)}'
     

In [17]:
print(view('styles.css', (1,10), nums=True))

     1 │ .cell {
     2 │   margin-bottom: 1rem;
     3 │ }
     4 │ 
     5 │ .cell > .sourceCode {
     6 │   margin-bottom: 0;
     7 │ }
     8 │ 
     9 │ .cell-output > pre {
    10 │   margin-bottom: 0;


In [18]:
#| exports
def create(path: str, # The path where the new file should be created
           file_text: str, # The text content to write to the new file
           overwrite:bool=False # Allows overwriting an existing file
          ) -> str:
    'Creates a new file with the given text content at the specified path'
    try:
        p = Path(path)
        if p.exists():
            if not overwrite: return f'Error: File already exists: {p}'
        p.parent.mkdir(parents=True, exist_ok=True)
        p.write_text(file_text)
        return f'Created file {p} containing:\n{file_text}'
    except Exception as e: return f'Error creating file: {str(e)}'     

In [19]:
print(create('test.txt', 'Hello, world!'))
print(view('test.txt', nums=True))

Created file test.txt containing:
Hello, world!
     1 │ Hello, world!


In [20]:
#| exports
def insert(path: str,  # The path to the file to modify
           insert_line: int, # The line number after which to insert the text (0 for beginning of file)
           new_str: str # The text to insert
          ) -> str: 
    'Insert text at a specific line number in a file.'
    try:
        p = Path(path)
        if not p.exists(): return f'Error: File not found: {p}'
            
        content = p.read_text().splitlines()
        if not (0 <= insert_line <= len(content)): return f'Error: Invalid line number {insert_line}'
            
        content.insert(insert_line, new_str)
        new_content = '\n'.join(content)
        p.write_text(new_content)
        return f'Inserted text at line {insert_line} in {p}.\nNew contents:\n{new_content}'
    except Exception as e: return f'Error inserting text: {str(e)}'

In [21]:
insert('test.txt', 0, 'Let\'s add a new line')
print(view('test.txt', nums=True))

     1 │ Let's add a new line
     2 │ Hello, world!


In [22]:
#| exports
def str_replace(path: str, # The path to the file to modify
                old_str: str, # The text to replace (must match exactly, including whitespace and indentation)
                new_str: str # The new text to insert in place of the old text
               ) -> str:
    'Replace a specific string in a file with a new string. This is used for making precise edits.'
    try:
        p = Path(path)
        if not p.exists(): return f'Error: File not found: {p}'
            
        content = p.read_text()
        count = content.count(old_str)
        
        if count == 0: return 'Error: Text not found in file'
        if count > 1: return f'Error: Multiple matches found ({count})'
            
        new_content = content.replace(old_str, new_str, 1)
        p.write_text(new_content)
        return f'Replaced text in {p}.\nNew contents:\n{new_content}'
    except Exception as e: return f'Error replacing text: {str(e)}'

In [23]:
str_replace('test.txt', 'new line', '')
print(view('test.txt', nums=True))

     1 │ Let's add a 
     2 │ Hello, world!


## Bash tool

The bash tool enables the model to execute shell commands in a persistent bash session, allowing system operations, script execution, and command-line automation.

**Overview**

The bash tool provides the model with:
- Persistent bash session that maintains state
- Ability to run any shell command
- Access to environment variables and working directory
- Command chaining and scripting capabilities

**Use cases**

- Development workflows: Run build commands, tests, and development tools
- System automation: Execute scripts, manage files, automate tasks
- Data processing: Process files, run analysis scripts, manage datasets
- Environment setup: Install packages, configure environments

**How it works**

The bash tool maintains a persistent session:
1. the model determines what command to run
2. you execute the command in a bash shell
3. reurn the output (stdout and stderr) to the model
4. session state persists between commands (environment variables, working directory)

**Parameters**

Parameter |	Required | Description
-- | -- | --
`command` | Yes* | The bash command to run
`restart` | No | Set to true to restart the bash session

*Required unless using `restart`

**Example: Multi-step automation**

The model can chain commands to complete complex tasks:

```bash
# User request
"Install the requests library and create a simple Python script that fetches a joke from an API, then run it."

# Model tool calls:
# 1. Install package
{"command": "pip install requests"}

# 2. Create script
{"command": "cat > fetch_joke.py << 'EOF'\nimport requests\nresponse = requests.get('https://official-joke-api.appspot.com/random_joke')\njoke = response.json()\nprint(f\"Setup: {joke['setup']}\")\nprint(f\"Punchline: {joke['punchline']}\")\nEOF"}

# 3. Run script
{"command": "python fetch_joke.py"}
```

The session maintains state between commands, so files created in step 2 are available in step 3.

**Handle errors**

Command execution timeout
- If a command takes too long to execute: "Error: Command timed out after 30 seconds"

Command not found
- If a command doesn't exist: "bash: nonexistentcommand: command not found"

Permission denied
- If there are permission issues: "bash: /root/sensitive-file: Permission denied"

**Implementation best practices**

Use command timeouts: Implement timeouts to prevent hanging commands.

Maintain session state: Keep the bash session persistent to maintain environment variables and working directory.

Handle large outputs: Truncate very large outputs to prevent token limit issues.

Log all commands: Keep an audit trail of executed commands.

Sanitize outputs: Remove sensitive information from command outputs.

**Security**

The bash tool provides direct system access. Implement these essential safety measures:
- Running in isolated environments (Docker/VM)
- Implementing command filtering and allowlists
- Setting resource limits (CPU, memory, disk)
- Logging all executed commands

Key recommendations
- Use ulimit to set resource constraints
- Filter dangerous commands (sudo, rm -rf, etc.)
- Run with minimal user permissions
- Monitor and log all command execution

**Common patterns**

Development workflows
- Running tests: pytest && coverage report
- Building projects: npm install && npm run build
- Git operations: git status && git add . && git commit -m "message"

File operations
- Processing data: wc -l *.csv && ls -lh *.csv
- Searching files: find . -name "*.py" | xargs grep "pattern"
- Creating backups: tar -czf backup.tar.gz ./data

System tasks
- Checking resources: df -h && free -m
- Process management: ps aux | grep python
- Environment setup: export PATH=PATH:/new/path && echo PATH

**Limitations**

- No interactive commands: Cannot handle vim, less, or password prompts
- No GUI applications: Command-line only
- Session scope: Persists within conversation, lost between API calls
- Output limits: Large outputs may be truncated
- No streaming: Results returned after completion

**Combining with other tools**

The bash tool is most powerful when combined with the text editor and other tools.

## Code execution tool

Code execution tool = bash tool + text editor tool **in a remote container**.

The code execution tool defined by Anthropic is implemented as a remote execution container, and most of its features are designed around the interaction between a local and remote cloud environment.

In our wordslab-notebooks context, we want to execute everything locally, so we will only keep a few interesting parts of the Anthropic documentation.

When this tool is provided, the model automatically gains access to two sub-tools:
- bash_code_execution: Run shell commands
- text_editor_code_execution: View, create, and edit files, including writing code

**How code execution works**

When you add the code execution tool to your API request:
1. The model evaluates whether code execution would help answer your question
2. The tool automatically provides the model with the following capabilities:
  - Bash commands: Execute shell commands for system operations and package management
  - File operations: Create, view, and edit files directly, including writing code
3. The model can use any combination of these capabilities in a single request
4. All operations run in a secure sandbox environment
5. The tool provides results with any generated charts, calculations, or analysis

**Containers**

The code execution tool runs in a secure, containerized environment designed specifically for code execution, with a higher focus on Python.

Runtime environment
- Python version: 3.11.12
- Operating system: Linux-based container
- Architecture: x86_64 (AMD64)

Resource limits
- Memory: 5GiB RAM
- Disk space: 5GiB workspace storage
- CPU: 1 CPU

Networking and security
- Internet access: Completely disabled for security
- External connections: No outbound network requests permitted
- Sandbox isolation: Full isolation from host system and other containers
- File access: Limited to workspace directory only
- Workspace scoping: Like Files, containers are scoped to the workspace of the API key
- Expiration: Containers expire 30 days after creation

Pre-installed libraries
- Data Science: pandas, numpy, scipy, scikit-learn, statsmodels
- Visualization: matplotlib, seaborn
- File Processing: pyarrow, openpyxl, xlsxwriter, xlrd, pillow, python-pptx, python-docx, pypdf, pdfplumber, pypdfium2, pdf2image, pdfkit, tabula-py, reportlab[pycairo], Img2pdf
- Math & Computing: sympy, mpmath
- Utilities: tqdm, python-dateutil, pytz, joblib, unzip, unrar, 7zip, bc, rg (ripgrep), fd, sqlite

Container reuse
- You can reuse an existing container across multiple API requests by providing the container ID from a previous response.
- This allows you to maintain created files between requests.

**How to use the tool**

Execute Bash commands

Ask the model to check system information and install packages: "Check the Python version and list installed packages"

Create and edit files directly

The model can create, view, and edit files directly in the sandbox using the file manipulation capabilities: "Create a config.yaml file with database settings, then update the port from 5432 to 3306"

Upload and analyze your own files

To analyze your own data files (CSV, Excel, images, etc.), upload them via the Files API and reference them in your request: "Analyze this CSV data"

```json
"content": [
                {"type": "text", "text": "Analyze this CSV data"},
                {"type": "container_upload", "file_id": "file_abc123"}
            ]
```

Retrieve generated files

When the tool creates files during code execution, you can retrieve these files using the Files API: "Create a matplotlib visualization and save it as output.png"
- Extract file IDs from the response
- Download the created files

Combine operations

A complex workflow using all capabilities:
- First, upload a file
- Extract file_id
- Then use it with code execution
- "Analyze this CSV data: create a summary report, save visualizations, and create a README with the findings"

**Response format**

The code execution tool can return two types of results depending on the operation:

Bash command response
```json
    "stdout": "total 24\ndrwxr-xr-x 2 user user 4096 Jan 1 12:00 .\ndrwxr-xr-x 3 user user 4096 Jan 1 11:00 ..\n-rw-r--r-- 1 user user  220 Jan 1 12:00 data.csv\n-rw-r--r-- 1 user user  180 Jan 1 12:00 config.json",
    "stderr": "",
    "return_code": 0
```

File operation responses

- View file
```json
    "file_type": "text",
    "content": "{\n  \"setting\": \"value\",\n  \"debug\": true\n}",
    "numLines": 4,
    "startLine": 1,
    "totalLines": 4
```

- Create file
  - is_file_update: whether file already existed
```json
    "is_file_update": false
```

- Edit file (str_replace)
  - lines: diff format
```json
    "oldStart": 3,
    "oldLines": 1,
    "newStart": 3,
    "newLines": 1,
    "lines": ["-  \"debug\": true", "+  \"debug\": false"]
```

**Errors**

Error codes by tool type:

| Tool          | Error Code                 | Description                                      |
|---------------|----------------------------|--------------------------------------------------|
| All tools     | unavailable                | The tool is temporarily unavailable              |
| All tools     | execution_time_exceeded    | Execution exceeded maximum time limit            |
| All tools     | container_expired          | Container expired and is no longer available     |
| All tools     | invalid_tool_input         | Invalid parameters provided to the tool          |
| All tools     | too_many_requests          | Rate limit exceeded for tool usage               |
| text_editor   | file_not_found             | File doesn't exist (for view/edit operations)    |
| text_editor   | string_not_found           | The old_str not found in file (for str_replace)  |
 
**Programmatic tool calling**

The code execution tool powers programmatic tool calling, which allows the model to write code that calls your custom tools programmatically within the execution container. This enables efficient multi-tool workflows, data filtering before reaching the model's context, and complex conditional logic.

Enable programmatic calling for your tools:
```json
    tools=[
        {
            "name": "get_weather",
            "description": "Get weather for a city",
            "input_schema": {...},
            "allowed_callers": ["code_execution_20250825"]  # Enable programmatic calling
        }
    ]
```

Learn more in the Programmatic tool calling documentation.

**Using code execution with Agent Skills**

The code execution tool enables the mdeol to use Agent Skills. Skills are modular capabilities consisting of instructions, scripts, and resources that extend the model's functionality.

Learn more in the Agent Skills documentation and Agent Skills API guide.

## Code interpreter

From https://github.com/AnswerDotAI/claudette

Code interpreter
Here is an example of using toolloop to implement a simple code interpreter with additional tools.


from toolslm.shell import get_shell
from fastcore.meta import delegates
import traceback
     

@delegates()
class CodeChat(Chat):
    imps = 'os, warnings, time, json, re, math, collections, itertools, functools, dateutil, datetime, string, types, copy, pprint, enum, numbers, decimal, fractions, random, operator, typing, dataclasses'
    def __init__(self, model: Optional[str] = None, ask:bool=True, **kwargs):
        super().__init__(model=model, **kwargs)
        self.ask = askm
        self.tools.append(self.run_cell)
        self.shell = get_shell()
        self.shell.run_cell('import '+self.imps)
     
We have one additional parameter to creating a CodeChat beyond what we pass to Chat, which is ask -- if that's True, we'll prompt the user before running code.


@patch
def run_cell(
    self:CodeChat,
    code:str,   # Code to execute in persistent IPython session
)->str:
    """Asks user for permission, and if provided, executes python `code` using persistent IPython session.
    Returns: Result of expression on last line (if exists); '#DECLINED#' if user declines request to execute"""
    confirm = f'Press Enter to execute, or enter "n" to skip?\n```\n{code}\n```\n'
    if self.ask and input(confirm): return '#DECLINED#'
    try: res = self.shell.run_cell(code)
    except Exception as e: return traceback.format_exc()
    return res.stdout if res.result is None else res.result

## Web pages

This tool is inspired by the library **ipykernel_helper** from **Answer.ai**. As of december 2025, this library is not open source, but it is available to users in the solve.it.com environment and is a dependency of other Apache 2.0 libraries, so I think it is OK to use it as an inspiration.

In [ ]:
#| export
def _absolutify_imgs(md, base_url):
    """This function rewrites Markdown image links so their URLs become absolute, using a base URL.
    - md: a Markdown string
    - base_url: the base URL used to resolve relative path
    """
    def fix(m):
        alt, img_url = m.group(1), m.group(2)
        if not img_url.startswith('http'): img_url = urljoin(base_url, img_url)
        return f'![{alt}]({img_url})'
    return re.sub(r'!\[(.*?)\]\((.*?)\)', fix, md)


In [ ]:
#| export
def _convert_math(soup, mode):
    """This function walks an HTML/XML document, finds MathML math elements, and replaces them with TeX/LaTeX markup suitable for text-based math renderers (like Markdown, MathJax, or KaTeX).
    - soup: a BeautifulSoup object representing parsed HTML/XML
    - mode: controls the inline math delimiter style
    Two inline math styles are supported:
    - mode='dollar': Inline math $x$ Display math $$x$$
    - other	Inline math \\(...\\) Display math $$x$$
    """
    for math in soup.find_all('math'):
        annot = math.find('annotation', {'encoding': 'application/x-tex'})
        if not annot:
            continue
        tex, display = annot.text.strip(), math.get('display') == 'block'
        if mode == 'dollar':
            wrap = f'$${tex}$$' if display else f'${tex}$'
        else:
            wrap = f'$${tex}$$' if display else f'\\({tex}\\)'
        math.replace_with(wrap)

In [ ]:
#| export
def scrape_url(url):
    "Get the html content of a web page using the cloudscraper library to bypass Cloudflare's anti-bot page."
    return create_scraper().get(url)

def read_url(url: str, as_md: bool = True, extract_section: bool = True, selector: str = None, math_mode: str = None):
    """This functions extracts a web page information for LLM ingestion
    1. Downloads a web page
    2. Parses HTML
    3. Optionally extracts a specific section (fragment or CSS selector)
    4. Converts MathML → LaTeX
    5. Optionally converts HTML → Markdown
    6. Convert code sections to fenced markdown blocks
    7. Makes image URLs absolute
    8. Returns the processed text
    """
    o = scrape_url(url)
    res, ctype = o.text, o.headers.get('content-type').split(';')[0]
    soup = BeautifulSoup(res, 'lxml')

    if selector:
        res = '\n\n'.join(str(s) for s in soup.select(selector))
    elif extract_section:
        parsed = urlparse(url)
        if parsed.fragment:
            section = soup.find(id=parsed.fragment)
            if section:
                elements = [section]
                current = section.next_sibling
                while current:
                    if hasattr(current, 'name') and current.name == section.name: break
                    elements.append(current)
                    current = current.next_sibling
                res = ''.join(str(el) for el in elements)
            else:
                res = ''
    else:
        res = str(soup)

    if math_mode:
        res_soup = BeautifulSoup(res, 'lxml')
        _convert_math(res_soup, math_mode)
        res = str(res_soup)

    if as_md and ctype == 'text/html':
        h = HTML2Text()
        h.body_width = 0
        # Handle code blocks
        h.mark_code = True
        res = h.handle(res)
        def _f(m): return f'```\n{dedent(m.group(1))}\n```'
        res = re.sub(r'\[code]\s*\n(.*?)\n\[/code]', _f, res or '', flags=re.DOTALL).strip()
        # Handle image urls
        res = _absolutify_imgs(res, urljoin(url, s['href'] if (s := soup.find('base')) else ''))
        # Handle math blocks
        if math_mode == 'safe':
            res = res.replace('\\\\(', '\\(').replace('\\\\)', '\\)')

    return res

In [ ]:
url2md = read_url("https://answerdotai.github.io/toolslm/")
url2md

'[ toolslm ](./index.html)\n\n__\n\n  1. [toolslm](./index.html)\n\n\n\n  * [ toolslm](./index.html)\n\n  * [ xml source](./xml.html)\n\n  * [ funccall source](./funccall.html)\n\n  * [ shell source](./shell.html)\n\n  * [ Download helpers](./download.html)\n\n  * [ Markdown Hierarchy Parser](./md_hier.html)\n\n\n\n\n## On this page\n\n  * Install\n  * How to use\n    * Context creation\n\n\n\n  * [__Report an issue](https://github.com/AnswerDotAI/toolslm/issues/new)\n\n\n\n## Other Formats\n\n  * [ __CommonMark](index.html.md)\n\n\n\n# toolslm\n\nTools to make language models a bit easier to use \n\nThis is a work in progress…\n\n## Install\n    \n    \n    pip install toolslm\n\n __\n\n## How to use\n\n### Context creation\n\ntoolslm has some helpers to make it easier to generate XML context from files, for instance [`folder2ctx`](https://AnswerDotAI.github.io/toolslm/xml.html#folder2ctx):\n    \n    \n    print(folder2ctx(\'samples\', prefix=False, file_glob=\'*.py\'))\n\n__\n    \n

In [ ]:
#| hide
from nbdev.showdoc import *

## Solveit tool exploration

Here is the list of unique Answerai library names sorted in alphabetical order:

### Librairies providing tools

| Library | GitHub Repo URL | Library Description |
|---------|-----------------|---------------------|
| **contextkit** | [*](https://github.com/AnswerDotAI/ContextKit "GitHub - AnswerDotAI/ContextKit: Useful LLM contexts ready to be used in AIMagic") https://github.com/AnswerDotAI/ContextKit | [*](https://github.com/AnswerDotAI/ContextKit "GitHub - AnswerDotAI/ContextKit: Useful LLM contexts ready to be used in AIMagic") Useful LLM contexts ready to be used in AIMagic. Library for gathering context from various sources (URLs, GitHub repos, Google Docs, PDFs, arXiv) for feeding to LLMs. |
| **dialoghelper** | [*](https://github.com/AnswerDotAI/dialoghelper "GitHub - AnswerDotAI/dialoghelper: Helper functions for solveit dialogs") https://github.com/AnswerDotAI/dialoghelper | [*](https://github.com/AnswerDotAI/dialoghelper "GitHub - AnswerDotAI/dialoghelper: Helper functions for solveit dialogs") Helper functions for solveit dialogs. Provides functions for message manipulation, gist management, screen capture, and tool integration in the solveit environment. |
| **execnb** | [*](https://github.com/AnswerDotAI/execnb "GitHub - AnswerDotAI/execnb: Execute a jupyter notebook, fast, without needing jupyter") https://github.com/AnswerDotAI/execnb | [*](https://github.com/AnswerDotAI/execnb "GitHub - AnswerDotAI/execnb: Execute a jupyter notebook, fast, without needing jupyter") Execute a jupyter notebook, fast, without needing jupyter. Provides CaptureShell for running code and capturing outputs without a Jupyter server. |
| **fastcore** | [*](https://github.com/AnswerDotAI "Answer.AI · GitHub") https://github.com/AnswerDotAI/fastcore | Python utilities and enhancements used across fast.ai projects. Includes foundational tools like `@patch`, `@delegates`, type dispatch, and LLM file/bash tools. |
| **ipykernel_helper** | PRIVATE REPO | Helper utilities for IPython/Jupyter kernels, providing `read_url`, `transient` display, enhanced completion, and namespace inspection for dialog environments. |
| **llms_txt** | [*](https://github.com/AnswerDotAI/llms-txt/blob/main/nbs/index.qmd "llms-txt/nbs/index.qmd at main · AnswerDotAI/llms-txt") https://github.com/AnswerDotAI/llms-txt | [*](https://github.com/AnswerDotAI/llms-txt/blob/main/nbs/index.qmd "llms-txt/nbs/index.qmd at main · AnswerDotAI/llms-txt") The `llms.txt` specification is open for community input. A GitHub repository hosts this informal overview. Tools for the /llms.txt standard helping language models use websites. |
| **playwrightnb** | https://github.com/AnswerDotAI/playwrightnb | Playwright browser automation integration for Jupyter notebooks, enabling headless browser control and web scraping. |
| **toolslm** | [*](https://github.com/AnswerDotAI/toolslm "GitHub - AnswerDotAI/toolslm: Tools to make language models a bit easier to use") https://github.com/AnswerDotAI/toolslm | [*](https://github.com/AnswerDotAI/toolslm "GitHub - AnswerDotAI/toolslm: Tools to make language models a bit easier to use") Tools to make language models a bit easier to use. Provides XML context creation, function schema generation, shell execution, and web content downloading for LLMs. |

### Summary: Answer.AI Tools for LLM Development

**LLM Tool Utilities**

| Library | Purpose | Key Functions |
|---------|---------|---------------|
| **fastcore.tools** | File/bash operations designed for LLM tool loops | `view`, `create`, `insert`, `str_replace`, `strs_replace`, `replace_lines`, `run_cmd`, `rg`, `sed` |
| **toolslm.funccall** | Function calling / tool use | `get_schema`, `call_func`, `call_func_async`, `python`, `mk_ns` |

**Context Preparation**

| Library | Purpose | Key Functions |
|---------|---------|---------------|
| **contextkit** | Gather context from various sources | `read_link`, `read_gh_repo`, `read_gh_file`, `read_gist`, `read_google_sheet`, `read_gdoc`, `read_dir`, `read_pdf`, `read_arxiv` |
| **toolslm.xml** | Convert files/folders to XML for LLMs | `folder2ctx`, `files2ctx`, `json_to_xml`, `docs_xml`, `nb2xml` |
| **toolslm.download** | Fetch web content for LLMs | `read_html`, `read_md`, `html2md`, `get_llmstxt`, `find_docs`, `read_docs` |
| **toolslm.md_hier** | Parse markdown hierarchically | `create_heading_dict`, `HeadingDict` |

**Message Formatting**

| Library | Purpose | Key Functions |
|---------|---------|---------------|
| **msglm** | Create properly formatted messages for LLM APIs | `mk_msg`, `mk_msgs`, `mk_msg_anthropic`, `mk_msg_openai`, `mk_ant_doc` (supports text, images, PDFs, caching) |

**Code Execution**

| Library | Purpose | Key Functions |
|---------|---------|---------------|
| **execnb** | Execute Jupyter notebooks without Jupyter | `CaptureShell`, `CaptureShell.run()`, `CaptureShell.execute()`, `read_nb`, `write_nb`, `new_nb` |
| **nbformat** | Notebook format handling & validation | `read`, `write`, `validate`, format conversion |
| **toolslm.shell** | Minimal IPython shell for code execution | `get_shell`, `run_cell` |

**Dialog/Session Management**

| Library | Purpose | Key Functions |
|---------|---------|---------------|
| **dialoghelper.core** | Interact with solveit dialogs | `read_msg`, `add_msg`, `update_msg`, `del_msg`, `run_msg`, `find_msgs`, `import_gist`, `url2note`, `ast_grep` |
| **dialoghelper.capture** | Screen sharing with AI | `setup_share`, `start_share`, `capture_screen`, `capture_tool` |

**Visual Overview**

```
┌─────────────────────────────────────────────────────────────────┐
│                    LLM Application Stack                        │
├─────────────────────────────────────────────────────────────────┤
│  USER INTERFACE                                                 │
│    dialoghelper (dialog management, screen capture)             │
├─────────────────────────────────────────────────────────────────┤
│  MESSAGE LAYER                                                  │
│    msglm (format messages for Claude/OpenAI/etc)                │
├─────────────────────────────────────────────────────────────────┤
│  CONTEXT LAYER                                                  │
│    contextkit (read from URLs, GitHub, Google, PDFs, arXiv)     │
│    toolslm.xml (convert files/folders to XML)                   │
│    toolslm.download (fetch & clean web content)                 │
│    toolslm.md_hier (parse markdown structure)                   │
├─────────────────────────────────────────────────────────────────┤
│  TOOL EXECUTION LAYER                                           │
│    fastcore.tools (file editing, bash commands)                 │
│    toolslm.funccall (function schemas, safe execution)          │
│    toolslm.shell (IPython execution)                            │
│    execnb (notebook execution)                                  │
├─────────────────────────────────────────────────────────────────┤
│  FORMAT LAYER                                                   │
│    nbformat (notebook format handling)                          │
└─────────────────────────────────────────────────────────────────┘
```

Key Themes

1. **Safety**: Tools return errors as strings for LLMs to debug; file operations have safeguards
2. **LLM-Optimized Formats**: XML for Claude, proper message structures for each API
3. **Single-Function Access**: Most operations are one function call (e.g., `read_gh_repo`, `folder2ctx`)
4. **Composability**: Libraries work together - contextkit gathers content, toolslm formats it, msglm packages it for APIs

### Details of the tools available in these libraries

#### contextkit

All functions follow th same design pattern: single required argument (location), optional parameters for customization, and return text or dict suitable for LLM consumption.

Web Content Functions

**`read_link(url, heavy=False, sel=None, useJina=False, ignore_links=False)`**
- Reads a URL and converts to markdown
- The heavy argument allows you to do a heavy scrape with a contactless browser using playwrightnb
- `sel` - CSS selector to extract specific content
- `useJina=True` - uses Jina.ai service for markdown conversion
- `ignore_links` - whether to strip out links

**`read_url(...)`** - Deprecated alias for `read_link()`

**`read_text(url)`**
- Get raw text from a URL (no markdown conversion)

**`read_html(url, sel=None, ...)`**
- Fetch URL, optionally select CSS elements, convert to clean markdown

GitHub Functions

**`read_gist(url)`**
- Returns raw gist content from a GitHub gist URL

**`read_gh_file(url)`**
- Reads contents of a single file from its GitHub URL

**`read_gh_repo(path_or_url, as_dict=False, verbose=False)`**
- Reads entire repo contents from GitHub URL, SSH address, or local path
- Clones/caches repos automatically
- Returns dict of `{filepath: content}` if `as_dict=True`, otherwise concatenated string

Google Services Functions

**`read_google_sheet(url)`**
- Reads a Google Sheet into CSV text

**`read_gdoc(url)`**
- Gets text content of a Google Doc converted to markdown

File System Functions

**`read_file(path)`**
- Returns file contents as string

**`read_dir(path, unicode_only=True, included_patterns=["*"], excluded_patterns=[".git/**"], verbose=False, as_dict=False)`**
- Reads files in a directory with glob pattern filtering
- `unicode_only=True` - skip binary files
- `included_patterns` - glob patterns to include
- `excluded_patterns` - glob patterns to exclude
- Returns dict or concatenated string with file markers

**`read_pdf(file_path)`**
- Extracts text from PDF using pypdf

Academic Functions

**`read_arxiv(url, save_pdf=False, save_dir='.')`**
- Get paper info from arXiv URL or ID
- Returns dict with title, authors, summary, published date, links
- Optionally downloads PDF and extracts LaTeX source
- Returns structured metadata perfect for LLM context

#### dialoghelper

dialoghelper is a library from Answer.AI providing helper functions for the solveit dialog environment. It enables programmatic interaction with dialog cells, message editing, gist management, and screen capture.

Module 1: **core** (`dialoghelper.core`)

The main module with functions for interacting with solveit dialogs:

Basics - Variable/Context Management

| Function | Description |
|----------|-------------|
| `find_var(var)` | Search for variable in all frames of the call stack |
| `set_var(var, val)` | Set variable value after finding it in call stack frames |
| `find_msg_id()` | Get current message ID from call stack (`__msg_id`) |
| `find_dname()` | Get current dialog name from call stack (`__dialog_id`) |
| `call_endp(path, ...)` | Call a solveit API endpoint |
| `curr_dialog(with_messages, dname)` | Get current dialog info |
| `msg_idx(msgid, dname)` | Get absolute index of message in dialog |

JavaScript/HTML Injection

| Function | Description |
|----------|-------------|
| `add_scr(scr, oob)` | Swap a script element into the DOM |
| `iife(code)` | Wrap JavaScript code in IIFE and execute via `add_html` |
| `add_html(content, dname)` | Send HTML to browser to be swapped into DOM (uses hx-swap-oob) |

Event System

| Function | Description |
|----------|-------------|
| `pop_data(idx, timeout)` | Pop data from a queue |
| `fire_event(evt, data)` | Fire a browser event |
| `event_get(evt, timeout, data)` | Fire event and wait for response data |

View/Edit Dialog Messages

| Function | Description |
|----------|-------------|
| `find_msgs(re_pattern, msg_type, limit, ...)` | Find messages matching regex/type in current dialog |
| `read_msg(n, relative, msgid, view_range, nums, ...)` | Get message by index (absolute or relative) with optional line range |
| `add_msg(content, placement, msgid, msg_type, ...)` | Add a new message (note/code/prompt) at specified position |
| `del_msg(msgid, dname)` | Delete a message |
| `update_msg(msgid, msg, content, ...)` | Update an existing message's content/properties |
| `run_msg(msgid, dname)` | Queue a message for execution |
| `url2note(url, ...)` | Read URL as markdown and add as note(s) below current message |

AST/Code Search

| Function | Description |
|----------|-------------|
| `ast_py(code)` | Get an ast-grep SgRoot node for Python code |
| `ast_grep(pattern, path, lang)` | Use ast-grep to find pattern in files |

Text Edit Functions (for editing message content)

| Function | Description |
|----------|-------------|
| `msg_insert_line(msgid, insert_line, new_str, ...)` | Insert text at specific line number |
| `msg_str_replace(msgid, old_str, new_str, ...)` | Replace first occurrence of string |
| `msg_strs_replace(msgid, old_strs, new_strs, ...)` | Replace multiple strings simultaneously |
| `msg_replace_lines(msgid, start_line, end_line, new_content, ...)` | Replace a range of lines |

Gist Management

| Function | Description |
|----------|-------------|
| `load_gist(gist_id)` | Retrieve a GitHub gist |
| `gist_file(gist_id)` | Get the first file from a gist |
| `import_string(code, name)` | Import code string as a module |
| `is_usable_tool(func)` | Check if function can be used as LLM tool (has docstring + typed params) |
| `mk_toollist(syms)` | Create markdown list of tools with `&` notation |
| `import_gist(gist_id, mod_name, ...)` | Import gist directly as module, optionally wildcard import |

Tool Info

| Function | Description |
|----------|-------------|
| `tool_info()` | Get information about available tools |
| `fc_tool_info()` | Get function-calling tool information |

Module 2: **capture** (`dialoghelper.capture`)

Screen capture functionality for sharing screens with the AI:

| Function | Description |
|----------|-------------|
| `setup_share()` | Setup screen sharing (initializes the capture system) |
| `start_share()` | Start screen sharing session |
| `capture_screen(timeout=15)` | Capture the screen as a PIL image |
| `capture_tool(timeout=15)` | Capture screen for LLM tool use - returns image data suitable for vision models |

dialoghelper is the infrastructure that powers the solveit interactive environment, enabling:
- **Dialog manipulation**: Add, edit, delete, search messages programmatically
- **Code execution**: Queue and run code cells
- **Text editing**: Line-based editing operations on message content
- **Gist integration**: Import tools/modules directly from GitHub gists
- **Screen capture**: Share your screen with the AI for visual understanding
- **AST search**: Search code using ast-grep patterns

#### execnb 

execnb is a fast.ai library for * * executing Jupyter notebooks without needing a Jupyter server or even having Jupyter installed.

Core execution:

* * CaptureShell - runs Jupyter code and captures notebook outputs
* * CaptureShell.run() - executes code strings and returns outputs
* * CaptureShell.execute() - executes a notebook and saves it with outputs filled in

### fastcore

fastcore provides helpful tools for running CLI commands and reading, modifying, and creating files in Python, primarily for AI's in tool loops for automating tasks involving the filesystem.

Bash Tools

**`run_cmd(cmd, argstr='', disallow_re=None, allow_re=None)`**
- Run cmd passing split argstr, optionally checking for allowed argstr
- Can include regex patterns to allow/disallow certain argument patterns (e.g., blocking parent directories)

**`rg(argstr, disallow_re=None, allow_re=None)`**
- Runs the ripgrep command for fast text searching across files
- Supports regex-based argument filtering for safety

**`sed(argstr, disallow_re=None, allow_re=None)`**
- Runs sed command for reading sections of files
- Useful for viewing specific line ranges with or without line numbers

Text Edit Tools

Python implementations of the text editor tools from Anthropic. These tools are especially useful in an AI's tool loop.

**`view(path, view_range=None, nums=False)`**
- View directory or file contents with optional line range and numbers
- Can specify 1-indexed line ranges and toggle line numbers

**`create(path, file_text, overwrite=False)`**
- Creates a new file with the given content at the specified path
- Safety feature: won't overwrite by default

**`insert(path, insert_line, new_str)`**
- Insert new_str at specified line number
- Uses 0-based indexing

**`str_replace(path, old_str, new_str)`**
- Replace first occurrence of old_str with new_str in file

**`strs_replace(path, old_strs, new_strs)`**
- Replace for each str pair in old_strs, new_strs
- Batch replacement of multiple string pairs

**`replace_lines(path, start_line, end_line, new_content)`**
- Replace lines in file using start and end line-numbers (index starting at 1)

Special LLM-Friendly Features

These tools have special behavior around errors. Since these have been specifically designed for work with LLMs, any exceptions created from their use is returned as a string to help them debug their work.
These tools are designed to be safe, predictable, and easy for LLMs to use in agentic workflows.

#### llms_txt

The llms-txt library provides a CLI and Python API to parse llms.txt files and create XML context files from them.

[*](https://www.analyticsvidhya.com/blog/2025/03/llms-txt/ "LLMs.txt Explained: The Web's New LLM-Ready Content Standard") [*](https://www.analyticsvidhya.com/blog/2025/03/llms-txt/ "LLMs.txt Explained: The Web's New LLM-Ready Content Standard") The llms_txt Python module provides the source code and helpers needed to create and use llms.txt files.

The llms-txt library provides:
1. **Parsing** - Convert llms.txt markdown files into structured Python data
2. **XML Generation** - Transform parsed data into LLM-friendly XML context (especially for Claude)
3. **CLI Tool** - Command-line utility for batch processing
4. **URL Fetching** - Automatically fetch and include content from linked URLs
Core Parsing Functions

**`parse_llms_file(txt, optional=True)`**
- [*](https://llmstxt.org/intro.html "Python module & CLI – llms-txt") [*](https://llmstxt.org/core.html "Python source – llms-txt") Parse llms.txt file contents to create a data structure with the sections of an llms.txt file
- [*](https://llmstxt.org/intro.html "Python module & CLI – llms-txt") Returns an AttrDict with keys: `title`, `summary`, `info`, `sections`
- `optional=True` - includes the optional section in the parsed output
- Example: `parsed.sections.Examples` returns list of links in the Examples section

**`parse_link(txt)`**
- [*](https://www.analyticsvidhya.com/blog/2025/03/llms-txt/ "LLMs.txt Explained: The Web's New LLM-Ready Content Standard") Extracts the title, URL, and optional description from a markdown link
- Returns dict with `title`, `url`, and `desc` keys

**Internal: `_parse_llms(txt)`**
- Lower-level parser that splits the file into start section and sectioned links
- Returns tuple of (start_text, sections_dict)

XML Context Generation

**`mk_ctx(d, optional=True, n_workers=None)`**
- [*](https://llmstxt.org/core.html "Python source – llms-txt") [*](https://llmstxt.org/core.html "Python source – llms-txt") Create a Project with a Section for each H2 part in the parsed data, optionally skipping the 'optional' section
- [*](https://llmstxt.org/core.html "Python source – llms-txt") For LLMs such as Claude, XML format is preferred
- Takes parsed llms.txt data structure and converts to XML
- `n_workers` - parallel workers for fetching URLs

**`create_ctx(txt, optional=True, n_workers=None)`**
- [*](https://llmstxt.org/intro.html "Python module & CLI – llms-txt") Create an LLM context file with XML sections, suitable for systems such as Claude (this is what the CLI calls behind the scenes)
- High-level function that parses and converts in one step

**`get_doc_content(url)`**
- [*](https://llmstxt.org/core.html "Python source – llms-txt") [*](https://llmstxt.org/core.html "Python source – llms-txt") Fetch content from local file if in nbdev repo
- Helper for retrieving document content from URLs or local paths

Command-Line Interface

**`llms_txt2ctx` (CLI command)**
- [*](https://llmstxt.org/intro.html "Python module & CLI – llms-txt") [*](https://llmstxt.org/intro.html "Python module & CLI – llms-txt") After installation, llms_txt2ctx is available in your terminal
- Usage: `llms_txt2ctx llms.txt > llms.md`
- [*](https://llmstxt.org/intro.html "Python module & CLI – llms-txt") [*](https://llmstxt.org/intro.html "Python module & CLI – llms-txt") Pass --optional True to add the 'optional' section of the input file
- [*](https://www.analyticsvidhya.com/blog/2025/03/llms-txt/ "LLMs.txt Explained: The Web's New LLM-Ready Content Standard") Uses the parsing helpers to process an llms.txt file and output an XML context file

File Format Specification

[*](https://llmstxt.org/core.html "Python source – llms-txt") [*](https://llmstxt.org/core.html "Python source – llms-txt") The llms.txt file spec contains: an H1 with the name of the project (the only required section), a blockquote with a short summary, zero or more markdown sections containing detailed information, and zero or more H2-delimited sections containing "file lists" of URLs
   
#### playwrightnb

[*](https://github.com/AnswerDotAI/playwrightnb "GitHub - AnswerDotAI/playwrightnb: Use sync mode Playwright interactively, inside a Jupyter notebook") [*](https://github.com/AnswerDotAI/playwrightnb "GitHub - AnswerDotAI/playwrightnb: Use sync mode Playwright interactively, inside a Jupyter notebook") playwrightnb provides quality-of-life helpers for interactive use of Playwright, particularly useful in Jupyter notebooks. It handles JavaScript rendering and other web complexities automatically.

Key Features

1. **Sync mode in Jupyter** - Use Playwright synchronously for interactive exploration
2. **Automatic iframe handling** - Returns both main content and iframe contents in a dict
3. **JavaScript support** - Handles dynamically loaded content automatically
4. **Stealth mode** - Optional bot detection avoidance
5. **CSS selectors** - Extract specific page sections easily
6. **Markdown conversion** - Built-in HTML to markdown conversion
7. **Flexible timeouts** - Configurable pause and timeout parameters

**Use cases:**
- Scraping JavaScript-heavy sites (Discord docs, dynamic help pages)
- Extracting content from iframes
- Interactive web exploration in notebooks
- Quick conversion of web pages to markdown for LLM context
    
Core Page Functions

**`get_page(*args, stealth=False, **kwargs)`**
- Get a Playwright page object
- `stealth=True` - uses playwright-stealth to avoid bot detection
- Returns an async page that can navigate to URLs

**`page_ready(page, pause=50, timeout=5000)`**
- Wait until main content of page is ready
- `pause` - milliseconds to wait (default: 50ms)
- `timeout` - maximum wait time in milliseconds (default: 5000ms)

**`frames_ready(page, pause=50, timeout=5000)`**
- Wait until all visible frames (if any) on page are ready
- Handles dynamically loaded iframes

**`wait_page(page, pause=50, timeout=5000)`**
- Wait until page and visible frames (if any) are ready
- Combines `page_ready` and `frames_ready`

**`get_full_content(page)`**
- Returns tuple of (page_content, iframes_dict)
- `iframes_dict` maps iframe IDs to their HTML contents

High-Level Reading Functions

**`read_page_async(url, pause=50, timeout=5000, stealth=False, page=None)`**
- [*](https://github.com/AnswerDotAI/playwrightnb "GitHub - AnswerDotAI/playwrightnb: Use sync mode Playwright interactively, inside a Jupyter notebook") Return contents of url and its iframes using Playwright async
- Handles JavaScript and dynamic content automatically
- Returns tuple of (main_content, iframes_dict)

**`read_page(url, pause=50, timeout=5000, stealth=False, page=None)`**
- [*](https://github.com/AnswerDotAI/playwrightnb "GitHub - AnswerDotAI/playwrightnb: Use sync mode Playwright interactively, inside a Jupyter notebook") Return contents of url and its iframes using Playwright (sync version)
- Same as `read_page_async` but synchronous for easier interactive use
- Perfect for Jupyter notebooks

HTML to Markdown Conversion

**`h2md(h)`**
- Convert HTML string to markdown using HTML2Text
- Simple utility for converting any HTML to markdown

**`url2md_async(url, sel=None, pause=50, timeout=5000, stealth=False, page=None)`**
- Read URL with `read_page_async`, optionally selecting CSS selector
- Converts result to markdown
- `sel` - CSS selector to extract specific content

**`url2md(url, sel=None, pause=50, timeout=5000, stealth=False, page=None)`**
- Read URL with `read_page` (sync version)
- Optionally select CSS selector and convert to markdown
- [*](https://github.com/AnswerDotAI/playwrightnb "GitHub - AnswerDotAI/playwrightnb: Use sync mode Playwright interactively, inside a Jupyter notebook") Great for accessing Discord's JS-rendered docs or other dynamic content

**`get2md(url, sel=None, params=None, headers=None, cookies=None, auth=None, proxy=None, follow_redirects=False, verify=True, timeout=5.0, trust_env=True)`**
- [*](https://github.com/AnswerDotAI/playwrightnb "GitHub - AnswerDotAI/playwrightnb: Use sync mode Playwright interactively, inside a Jupyter notebook") Read URL with httpx.get (no JavaScript rendering)
- Faster alternative when you don't need JS rendering
- Supports all standard HTTP options (headers, auth, proxies, etc.)
- Optionally select CSS content with `sel`

#### toolslm

toolslm provides tools to make language models easier to use, focused on context creation and XML handling:

**Context Creation Functions:**
- `folder2ctx` - generates XML context from files, useful for providing file contents to LLMs
- `json_to_xml` - converts JSON/dict structures to XML format

**Key modules** (based on the repo structure):
- `xml` - XML handling utilities
- `funccall` - Function calling support
- `shell` - Shell command integration
- `download` - Download utilities
- `md_hier` - Markdown hierarchy handling

The library is designed to help structure information (especially from codebases and files) into XML format that works well with Claude and other LLMs.

1. **xml** (`toolslm.xml`)

Provides functions for converting content to XML format optimized for LLMs (especially Claude/Anthropic):

| Function | Description |
|----------|-------------|
| `json_to_xml(d, rnm)` | Convert a JSON/dict to XML with a root name |
| `mk_doctype(content, src)` | Create a named tuple with source and content |
| `mk_doc(index, content, src)` | Create a single document in Anthropic's recommended XML format |
| `docs_xml(docs, srcs, prefix, details)` | Create XML string with multiple documents in Anthropic's format |
| `read_file(fname)` | Read file content, converting notebooks to XML if needed |
| `files2ctx(fnames, prefix)` | Convert multiple files to XML context |
| `folder2ctx(folder, prefix, **kwargs)` | Generate XML context from all files in a folder |
| `folder2ctx_cli` | Command-line interface for `folder2ctx` |
| `nb2xml(fname)` | Convert a Jupyter notebook to XML |
| `cell2xml(cell)` | Convert a notebook cell to XML format |
| `cell2out(o)` | Convert notebook output to XML format |

2. **funccall** (`toolslm.funccall`)

Tools for function calling / tool use with LLMs:

| Function | Description |
|----------|-------------|
| `get_schema(f)` | Generate JSON schema for a class, function, or method (for tool definitions) |
| `call_func(fc_name, fc_inputs, ns)` | Call a function by name with inputs using a namespace |
| `call_func_async(fc_name, fc_inputs, ns)` | Async version of `call_func` |
| `python(code, glb, loc, timeout)` | Execute Python code with timeout, returning final expression (like IPython) |
| `mk_ns(fs)` | Create a namespace dict from functions or dicts |
| `PathArg(path)` | Helper for filesystem path arguments |

3. **shell** (`toolslm.shell`)

Provides a minimal IPython shell for code execution:

| Function | Description |
|----------|-------------|
| `get_shell()` | Get a `TerminalInteractiveShell` with minimal functionality (no logging, history, automagic) |
| `run_cell(cell, timeout)` | Patched method to run cells with timeout and output capture |

4. **download** (`toolslm.download`)

Functions for fetching and processing web content for LLM consumption:

| Function | Description |
|----------|-------------|
| `clean_md(text, rm_comments, rm_details)` | Remove HTML comments and `<details>` sections from markdown |
| `read_md(url, ...)` | Read markdown from URL and clean it |
| `html2md(s, ignore_links)` | Convert HTML string to markdown |
| `read_html(url, sel, multi, wrap_tag, ...)` | Fetch URL, optionally select CSS elements, convert to clean markdown |
| `get_llmstxt(url, optional, n_workers)` | Get and expand an `llms.txt` file (LLM-friendly site documentation) |
| `split_url(url)` | Split URL into base, path, and filename |
| `find_docs(url)` | Find LLM-friendly docs (llms.txt or markdown) from a URL |
| `read_docs(url, ...)` | Read and return LLM-friendly documentation for a URL |

5. **md_hier** (`toolslm.md_hier`)

Parse markdown into a hierarchical dictionary structure:

| Function/Class | Description |
|----------------|-------------|
| `HeadingDict` | A dict subclass that also stores the original markdown text in `.text` |
| `create_heading_dict(text, rm_fenced)` | Parse markdown into nested dict based on heading hierarchy (e.g., `result['Section']['Subsection'].text`) |

This is useful for navigating and extracting specific sections from markdown documents programmatically.

**Summary**: toolslm provides a complete toolkit for preparing context for LLMs - converting files/folders to XML, generating function schemas for tool calling, executing code safely, fetching web documentation, and parsing markdown hierarchically.
    
#### msglm - Message Creation for LLMs

msglm makes it easier to create messages for language models like Claude and OpenAI GPTs.

**Text messages:**
- `mk_msgs` - takes a list of strings and an api format (e.g. "openai") and generates the correct format
- `mk_msg` - creates a single message

**API-specific wrappers:**
- `mk_msg_anthropic` / `mk_msgs_anthropic` - for Anthropic
- `mk_msg_openai` / `mk_msgs_openai` - for OpenAI

**Advanced features:**
- Image chats - pass raw image bytes in a list with your question
- Prompt caching - pass cache=True to mk_msg or mk_msgs (Anthropic)
- PDF support - pass raw pdf bytes just like image chats (Anthropic beta)
- `mk_ant_doc` - creates document objects for Anthropic citations feature

**Summary:** toolslm helps structure context (files, XML), while msglm helps format the actual message payloads for different LLM APIs, handling text, images, PDFs, and advanced features like caching and citations.


### Librairies using tools

| Library | GitHub Repo URL | Library Description |
|---------|-----------------|---------------------|
| **claudette** | [*](https://github.com/AnswerDotAI/claudette "GitHub - AnswerDotAI/claudette: Claudette is Claude's friend") [*](https://github.com/AnswerDotAI/claudette "GitHub - AnswerDotAI/claudette: Claudette is Claude's friend") [*](https://github.com/AnswerDotAI/claudette "GitHub - AnswerDotAI/claudette: Claudette is Claude's friend") https://github.com/AnswerDotAI/claudette | [*](https://github.com/AnswerDotAI/claudette "GitHub - AnswerDotAI/claudette: Claudette is Claude's friend") Wrapper for Anthropic's Python SDK. The SDK works well, but is quite low level. Claudette automates pretty much everything that can be automated, whilst providing full control. |
| **cosette** | [*](https://github.com/AnswerDotAI/cosette "GitHub - AnswerDotAI/cosette: Claudette's sister, a helper for OpenAI GPT") https://github.com/AnswerDotAI/cosette | [*](https://github.com/AnswerDotAI/cosette "GitHub - AnswerDotAI/cosette: Claudette's sister, a helper for OpenAI GPT") Claudette's sister, a helper for OpenAI GPT. High-level wrapper for OpenAI's SDK with stateful chat, tool calling, and streaming support. |
| **lisette** | https://github.com/AnswerDotAI/lisette | LiteLLM helper providing unified access to 100+ LLM providers using the OpenAI API format, with stateful chat, tools, web search, and reasoning support. |
| **msglm** | https://github.com/AnswerDotAI/msglm | Makes it easier to create messages for LLMs (Claude, OpenAI). Handles text, images, PDFs, prompt caching, and API-specific formatting. |

### Development tools

| Library | GitHub Repo URL | Library Description |
|---------|-----------------|---------------------|
| **fastcaddy** | https://github.com/AnswerDotAI/fastcaddy | Python wrapper for Caddy web server integration, simplifying HTTPS setup and reverse proxy configuration for FastHTML apps. |
| **fastlite** | [*](https://github.com/AnswerDotAI "Answer.AI · GitHub") https://github.com/AnswerDotAI/fastlite | A thin, expressive wrapper around SQLite for Python, providing easy database operations with minimal boilerplate. |
| **fastlucide** | https://github.com/AnswerDotAI/fastlucide | Lucide icons integration for FastHTML applications, providing SVG icons as Python components. |
| **fasthtml** | https://github.com/AnswerDotAI/fasthtml | Modern Python web framework for building HTML applications with HTMX and Starlette, using pure Python for HTML generation. |
| **ghapi** | https://github.com/AnswerDotAI/ghapi | [*](https://github.com/orgs/AnswerDotAI/repositories "AnswerDotAI repositories · GitHub") A delightful and complete interface to GitHub's amazing API. Auto-generated Python client covering all GitHub API endpoints. |
| **monsterui** | [*](https://github.com/AnswerDotAI "Answer.AI · GitHub") https://github.com/AnswerDotAI/MonsterUI | [*](https://github.com/AnswerDotAI "Answer.AI · GitHub") MonsterUI is a FastHTML Tailwind-powered UI framework for building beautiful web interfaces with minimal code. |
| **nbdev** | [*](https://github.com/AnswerDotAI "Answer.AI · GitHub") https://github.com/AnswerDotAI/nbdev | Literate programming framework that allows writing library code, tests, and documentation in Jupyter Notebooks, then exporting to Python modules. |
| **shell_sage** | https://github.com/AnswerDotAI/shell_sage | [*](https://github.com/orgs/AnswerDotAI/repositories "AnswerDotAI repositories · GitHub") ShellSage saves sysadmins' sanity by solving shell script snafus super swiftly. AI-powered shell command helper. |

## Tools catalog

Here is the list of tools we want to develop.

### Enhance IPython kernel

#### ipykernel_helper

ipykernel_helper is a library that provides helper utilities for working with IPython/Jupyter kernels. 

It appears to be part of the infrastructure that makes the Dialog Engineering environment work smoothly, providing convenient functions that are useful in an interactive coding environment.

**IPython InteractiveShell - Complete Feature Summary**

`InteractiveShell` is the core class of IPython, providing an enhanced, interactive Python environment. It's a singleton class that manages everything from code execution to history, completion, and output formatting.

1. **Code Execution**

| Method | Description |
|--------|-------------|
| `run_cell(raw_cell, store_history, silent, shell_futures, cell_id)` | Run a complete IPython cell including magics |
| `run_cell_async(...)` | Async version of `run_cell` |
| `run_code(code_obj, result, async_)` | Execute a compiled code object |
| `run_ast_nodes(nodelist, cell_name, interactivity, compiler, result)` | Run a sequence of AST nodes |
| `run_line_magic(magic_name, line)` | Execute a line magic like `%timeit` |
| `run_cell_magic(magic_name, line, cell)` | Execute a cell magic like `%%bash` |
| `safe_execfile(fname, *where, ...)` | Safely execute a .py file |
| `safe_execfile_ipy(fname, ...)` | Execute .ipy or .ipynb files with IPython syntax |
| `safe_run_module(mod_name, where)` | Safe version of `runpy.run_module()` |

2. **Namespace Management**

| Method/Property | Description |
|-----------------|-------------|
| `user_ns` | The user's namespace dictionary |
| `user_ns_hidden` | Hidden namespace items (not shown to user) |
| `all_ns_refs` | List of all namespace dictionaries where objects may be stored |
| `push(variables, interactive)` | Inject variables into user namespace |
| `del_var(varname, by_name)` | Delete a variable from namespaces |
| `drop_by_id(variables)` | Remove variables if they match given values |
| `ev(expr)` | Evaluate Python expression in user namespace |
| `ex(cmd)` | Execute Python statement in user namespace |
| `init_user_ns()` | Initialize user-visible namespaces to defaults |
| `reset(new_session, aggressive)` | Clear all internal namespaces |
| `reset_selective(regex)` | Clear variables matching a regex pattern |

3. **Code Completion**

| Method | Description |
|--------|-------------|
| `init_completer()` | Initialize completion machinery |
| `complete(text, line, cursor_pos)` | Return completed text and list of completions |
| `set_completer_frame(frame)` | Set the frame for completion context |
| `set_custom_completer(completer, pos)` | Add a custom completer function |

4. **Code Transformation**

| Method/Property | Description |
|-----------------|-------------|
| `transform_cell(raw_cell)` | Transform input cell before parsing (handles `%magic`, `!system`, etc.) |
| `transform_ast(node)` | Apply AST transformations from `ast_transformers` |
| `input_transformers_post` | List of string transformers applied after IPython's own |
| `ast_transformers` | List of `ast.NodeTransformer` instances for code modification |
| `check_complete(code)` | Check if code is ready to execute or needs continuation |

5. **History Management**

| Method/Property | Description |
|-----------------|-------------|
| `init_history()` | Setup command history and autosaves |
| `extract_input_lines(range_str, raw)` | Return input history slices as string |
| `history_length` | Total length of command history |
| `history_load_length` | Number of entries loaded at startup |

6. **Magic System**

| Method | Description |
|--------|-------------|
| `find_magic(magic_name, magic_kind)` | Find and return a magic by name |
| `find_line_magic(magic_name)` | Find a line magic |
| `find_cell_magic(magic_name)` | Find a cell magic |
| `register_magic_function(func, magic_kind, magic_name)` | Expose a function as a magic |
| `define_macro(name, themacro)` | Define a new macro |

7. **Object Inspection**

| Method | Description |
|--------|-------------|
| `object_inspect(oname, detail_level)` | Get object info |
| `object_inspect_mime(oname, detail_level, omit_sections)` | Get object info as mimebundle |
| `object_inspect_text(oname, detail_level)` | Get object info as formatted text |
| `find_user_code(target, raw, py_only, ...)` | Get code from history, file, URL, or string |

8. **Error Handling & Debugging**

| Method/Property | Description |
|-----------------|-------------|
| `showtraceback(exc_tuple, filename, tb_offset, ...)` | Display exception that just occurred |
| `showsyntaxerror(filename, running_compiled_code)` | Display syntax error |
| `showindentationerror()` | Handle IndentationError |
| `get_exception_only(exc_tuple)` | Get exception string without traceback |
| `set_custom_exc(exc_tuple, handler)` | Set custom exception handler |
| `excepthook(etype, value, tb)` | Custom sys.excepthook |
| `debugger(force)` | Call pdb debugger |
| `call_pdb` | Control auto-activation of pdb at exceptions |
| `pdb` | Automatically call pdb after every exception |
| `xmode` | Switch modes for exception handlers |

9. **System Interaction**

| Method | Description |
|--------|-------------|
| `system(cmd)` | Call cmd in subprocess, piping stdout/err |
| `system_piped(cmd)` | Call cmd with piped output |
| `system_raw(cmd)` | Call cmd using os.system or subprocess |
| `getoutput(cmd, split, depth)` | Get output from subprocess |

10. **Expression Evaluation**

| Method | Description |
|--------|-------------|
| `user_expressions(expressions)` | Evaluate dict of expressions, return rich mime-typed display_data |
| `var_expand(cmd, depth, formatter)` | Expand Python variables in a string |

11. **Async Support**

| Property/Method | Description |
|-----------------|-------------|
| `autoawait` | Automatically run await in top-level REPL |
| `should_run_async(raw_cell, ...)` | Determine if cell should run via coroutine runner |
| `loop_runner` | Select the loop runner for async code |

12. **Configuration Properties**

| Property | Description |
|----------|-------------|
| `autocall` | Auto-call callable objects (0=off, 1=smart, 2=full) |
| `autoindent` | Auto-indent code |
| `automagic` | Call magics without leading `%` |
| `colors` | Color scheme (nocolor, neutral, linux, lightbg) |
| `cache_size` | Output cache size (default 1000) |
| `ast_node_interactivity` | Which nodes display output ('all', 'last', 'last_expr', 'none', 'last_expr_or_assign') |
| `banner1` / `banner2` | Banner text before/after profile |
| `display_page` | Show pager content as regular output |
| `enable_html_pager` | Enable HTML in pagers |
| `enable_tip` | Show tip on IPython start |
| `show_rewritten_input` | Show rewritten input for autocall |
| `sphinxify_docstring` | Enable rich HTML docstrings |
| `warn_venv` | Warn if running in venv without IPython installed |

**Architecture Summary**

```
┌─────────────────────────────────────────────────────────────────────┐
│                    InteractiveShell (Singleton)                     │
├─────────────────────────────────────────────────────────────────────┤
│  NAMESPACES                                                         │
│    user_ns, user_ns_hidden, all_ns_refs                            │
├─────────────────────────────────────────────────────────────────────┤
│  CODE EXECUTION PIPELINE                                            │
│    raw_cell → transform_cell → parse → transform_ast → run_code    │
├─────────────────────────────────────────────────────────────────────┤
│  SUBSYSTEMS                                                         │
│    • Completer (tab completion)                                     │
│    • History Manager                                                │
│    • Magic System (line/cell magics)                               │
│    • Display Publisher                                              │
│    • Exception Handler / Debugger                                   │
│    • Input Transformers (magics, shell commands)                   │
│    • AST Transformers                                               │
├─────────────────────────────────────────────────────────────────────┤
│  EXTENSIONS                                                         │
│    • Hooks (set_hook)                                               │
│    • Custom completers (set_custom_completer)                      │
│    • Custom exception handlers (set_custom_exc)                    │
│    • Magic functions (register_magic_function)                     │
└─────────────────────────────────────────────────────────────────────┘
```

**How ipykernel_helper Extends InteractiveShell**

The `ipykernel_helper` library uses fastcore's `@patch` to add methods to `InteractiveShell`:

| Added Method | Purpose |
|--------------|---------|
| `user_items()` | Get user-defined vars & functions (filtered) |
| `get_vars(vs)` | Get specific variable values |
| `get_schemas(fs)` | Get JSON schemas for functions (tool calling) |
| `ranked_complete(code, ...)` | Enhanced completion with ranking |
| `sig_help(code, ...)` | Signature help using Jedi |
| `xpush(**kw)` | Push with kwargs |
| `publish(data, ...)` | Enhanced display publishing |

This is how solveit extends IPython's core to support AI-assisted coding!

#### Extract information from the user's namespace

These are methods patched onto IPython's InteractiveShell by ipykernel_helper. 

Getting an InteractiveShell instance from a notebook code cell is straightforward.

```python
from IPython import get_ipython
shell = get_ipython()
```

They help extract information from the user's namespace for use in dialog/AI contexts.

```python
InteractiveShell.user_items(self, max_len=200, xtra_skip=())
```

Purpose: Get user-defined variables and functions from the IPython namespace, filtering out system/internal items.

Returns: Tuple of (user_vars, user_fns)
- user_vars: Dict of {var_name: repr_string} - variable names and their string representations (truncated to max_len)
- user_fns: Dict of {func_name: signature_string} - user-defined function names and their signatures

What it filters out:
- Hidden namespace items (user_ns_hidden)
- Items starting with _
- Types, modules, methods, builtins, typing constructs
- Specific names like nbmeta, receive_nbmeta

Use case: Let the AI know what variables and functions the user has defined in their session.

```python
InteractiveShell.get_vars(self:InteractiveShell, vs:list, literal=True)
```

Purpose: Retrieve specific variable values from the namespace

Parameters:
- vs: List of variable names to retrieve
- literal: If True, try to return actual Python literals; if False, return string representation

Returns: 
- Dict of {var_name: value} for variables that exist in the namespace

The literal=True behavior:
- Tries to repr() the value, then literal_eval() it back
- If that succeeds, returns the actual value (works for dicts, lists, strings, numbers, etc.)
- If it fails (complex objects), returns the string representation instead

Use case: When the user shares specific variables with the AI using `$varname` notation, this function retrieves their current values.

```python
InteractiveShell.get_schemas(self:InteractiveShell, fs:list)
```

Purpose: Get JSON schemas for functions (for LLM tool calling).

Parameters:
- fs: List of function names to get schemas for

Returns: Dict of {func_name: schema_or_error}
- If successful: {'type': 'function', 'function': {JSON schema}}
- If not found: "funcname not found. Did you run it?"
- If schema error: "funcname: error message"

The schema format (via `toolslm.funccall.get_schema`):

```json
{
    'type': 'function',
    'function': {
        'name': 'my_func',
        'description': 'Docstring here',
        'parameters': {
            'type': 'object',
            'properties': {
                'param1': {'type': 'string', 'descmription': '...'},
                'param2': {'type': 'integer', 'description': '...'}
            },
            'required': ['param1']
        }
    }
}
```

Use case: When the user shares tools with the AI using `&toolname` notation, this function generates the JSON schemas that tell the AI how to call those functions.

#### Solveit use of get_vars() and get_schema()

The `$varname` and `&toolname` Notation System

Overview

When you write a prompt in solveit containing `$myvar` or `&mytool`, the system:
1. **Parses** your prompt for these special notations
2. **Extracts** the referenced names
3. **Retrieves** their values/schemas from the Python namespace
4. **Injects** this information into the context sent to the AI

**Variable Sharing: `$varname`**

What Happens

```
User writes prompt:
┌─────────────────────────────────────────────┐
│ "Analyze the data in `$df` and tell me     │
│  the average of the 'sales' column"         │
└─────────────────────────────────────────────┘
                    │
                    ▼
            Parse for $-prefixed names
                    │
                    ▼
            Found: ['df']
                    │
                    ▼
            Call: shell.get_vars(['df'])
                    │
                    ▼
            Retrieve current value of df
                    │
                    ▼
┌─────────────────────────────────────────────┐
│ Context sent to AI includes:                │
│                                             │
│ <variables>                                 │
│   df = DataFrame with 100 rows, columns:    │
│        ['date', 'sales', 'region', ...]     │
│ </variables>                                │
│                                             │
│ User prompt: "Analyze the data in df..."    │
└─────────────────────────────────────────────┘
```

Key Behavior

1. Current value: The variable's value at the time the prompt is sent (not when it was first defined)
2. Retroactive updates: If you change the variable and re-run, the context updates
3. Safe representation: Uses `_safe_repr()` to truncate large values (default 200 chars)
4. Literal conversion: Tries to preserve actual Python types when possible via `literal_eval`

Example Flow

```python
# Cell 1: Define data
df = pd.DataFrame({'a': [1,2,3], 'b': [4,5,6]})

# Cell 2: Prompt with $df
# "What's the sum of column 'a' in `$df`?"

# Behind the scenes:
shell.get_vars(['df'])
# Returns: {'df': "   a  b\n0  1  4\n1  2  5\n2  3  6"}
```

**Tool Sharing: `&toolname`**

What Happens

```
User writes prompt:
┌─────────────────────────────────────────────┐
│ "Use `&calculate` to add 15 and 27"         │
└─────────────────────────────────────────────┘
                    │
                    ▼
            Parse for &-prefixed names
                    │
                    ▼
            Found: ['calculate']
                    │
                    ▼
            Call: shell.get_schemas(['calculate'])
                    │
                    ▼
            Generate JSON schema from function
                    │
                    ▼
┌─────────────────────────────────────────────┐
│ AI receives tool definition:                │
│                                             │
│ {                                           │
│   "type": "function",                       │
│   "function": {                             │
│     "name": "calculate",                    │
│     "description": "Add two numbers",       │
│     "parameters": {                         │
│       "type": "object",                     │
│       "properties": {                       │
│         "a": {"type": "integer"},           │
│         "b": {"type": "integer"}            │
│       },                                    │
│       "required": ["a", "b"]                │
│     }                                       │
│   }                                         │
│ }                                           │
└─────────────────────────────────────────────┘
```

**Requirements for Tools**

From `dialoghelper.core.is_usable_tool()`, a function must have:
1. Type annotations for all parameters
2. A docstring (becomes the tool description)

```python
# ✅ Valid tool
def calculate(a: int, b: int) -> int:
    "Add two numbers together"
    return a + b

# ❌ Not usable - no docstring
def bad_tool(a: int, b: int) -> int:
    return a + b

# ❌ Not usable - missing type hints
def another_bad(a, b):
    "Add numbers"
    return a + b
```

**Tool Execution Flow**

When the AI decides to use a tool:

```
┌─────────────────────────────────────────────┐
│ AI Response:                                │
│ "I'll use calculate to add those numbers"   │
│                                             │
│ Tool Call:                                  │
│   name: "calculate"                         │
│   arguments: {"a": 15, "b": 27}             │
└─────────────────────────────────────────────┘
                    │
                    ▼
            solveit receives tool call
                    │
                    ▼
            Look up 'calculate' in namespace
                    │
                    ▼
            Execute: calculate(a=15, b=27)
                    │
                    ▼
            Result: 42
                    │
                    ▼
┌─────────────────────────────────────────────┐
│ Tool result sent back to AI                 │
│                                             │
│ AI continues: "The sum of 15 and 27 is 42"  │
└─────────────────────────────────────────────┘
```

**The Parsing Mechanism**

The parsing likely uses regex to find these patterns:

```python
import re

def parse_prompt(prompt):
    # Find $varname in backticks
    var_pattern = r'`\$(\w+)`'
    vars_found = re.findall(var_pattern, prompt)
    
    # Find &toolname in backticks
    tool_pattern = r'`&(\w+)`'
    tools_found = re.findall(tool_pattern, prompt)
    
    return vars_found, tools_found

# Example
prompt = "Use `&calculate` on `$x` and `$y`"
parse_prompt(prompt)
# Returns: (['x', 'y'], ['calculate'])
```

**Complete Architecture**

```
┌─────────────────────────────────────────────────────────────────────┐
│                      USER WRITES PROMPT                             │
│  "Use `&my_tool` to process `$my_data` and return the result"      │
└─────────────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────────────┐
│                      SOLVEIT FRONTEND                               │
│  1. Parse prompt for `$name` and `&name` patterns                  │
│  2. Extract: vars=['my_data'], tools=['my_tool']                   │
│  3. Request from kernel: get_vars(), get_schemas()                 │
└─────────────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────────────┐
│                      IPYKERNEL (Python)                             │
│  shell.get_vars(['my_data'])                                       │
│    → {'my_data': [1, 2, 3, 4, 5]}                                  │
│                                                                     │
│  shell.get_schemas(['my_tool'])                                    │
│    → {'my_tool': {'type': 'function', 'function': {...}}}          │
└─────────────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────────────┐
│                      CONTEXT ASSEMBLY                               │
│  <variables>                                                        │
│    my_data = [1, 2, 3, 4, 5]                                       │
│  </variables>                                                       │
│                                                                     │
│  Tools: [my_tool schema]                                           │
│                                                                     │
│  Prompt: "Use my_tool to process my_data..."                       │
└─────────────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────────────┐
│                      AI (Claude/etc)                                │
│  - Sees variable values in context                                 │
│  - Has tool schemas available                                      │
│  - Can call tools, results executed in user's kernel               │
└─────────────────────────────────────────────────────────────────────┘
```

**Key Points**

| Aspect | `$varname` | `&toolname` |
|--------|-----------|-------------|
| Purpose | Share variable value | Share function as callable tool |
| Retrieval | `get_vars()` | `get_schemas()` |
| What AI sees | Current value (as string/literal) | JSON schema for tool calling |
| Requirements | Variable must exist | Function needs types + docstring |
| AI can modify? | No (read-only snapshot) | Yes (can call the function) |

**Practical Tips**

1. Variables are snapshots: The AI sees the value when you send the prompt
2. Tools execute in your kernel: Side effects happen in your session
3. Use backticks: The notation is `` `$var` `` and `` `&tool` ``, not bare `$var`
4. Check tool validity: Use `is_usable_tool(func)` to verify a function can be used as a tool
5. List tools: Use `mk_toollist(symbols)` from dialoghelper to see available tools formatted for prompts

#### Customize Jupyterlab default behavior to enhance variables display

Let me explain how JupyterLab's Debugger panel works and whether you could hook into it.

**How JupyterLab's Debugger Panel Gets Variables**

The Debugger panel in JupyterLab uses a completely different mechanism than these ipykernel_helper functions:
1. Debug Adapter Protocol (DAP) - JupyterLab's debugger communicates via the DAP standard
2. debugpy - The Python debugger backend 
3. Kernel debug messages - Special debug_request/debug_reply messages over ZMQ
The flow looks like:

```
JupyterLab Debugger UI
        │
        ▼ (DAP protocol)
    debugpy 
        │
        ▼ (inspects frames)
    Python runtime
```
    
**The Challenge**

The user_items(), get_vars(), get_schemas() functions are:
- Pull-based: Called on demand by the solveit frontend
- Custom protocol: Use transient display messages or custom endpoints
- Namespace-focused: Look at InteractiveShell.user_ns

JupyterLab's Debugger is:
- DAP-based: Uses standardized debug adapter protocol
- Frame-focused: Inspects stack frames, not just the namespace
- debugpy-controlled: The variable inspection happens inside debugpy

**Option: Custom Variable Inspector Extension**

Instead of modifying the Debugger, create a JupyterLab extension that:
- Sends custom comm messages to the kernel
- Calls user_items() or similar
- Displays results in a custom panel

```python
# Kernel side
from ipykernel.comm import Comm

def send_variables():
    shell = get_ipython()
    user_vars, user_fns = shell.user_items()
    comm = Comm(target_name='variable_inspector')
    comm.send({'vars': user_vars, 'fns': user_fns})
```

**Practical Recommendation**

The easiest approach that gives you similar functionality to solveit's variable display in standard JupyterLab:

```python
# In your notebook or startup file
from IPython import get_ipython
from IPython.display import display, HTML
import json

def show_user_items(max_len=100):
    """Display user variables and functions like solveit does"""
    shell = get_ipython()
    user_vars, user_fns = shell.user_items(max_len=max_len)
    
    html = "<details open><summary><b>Variables</b></summary><ul>"
    for k, v in user_vars.items():
        html += f"<li><code>{k}</code>: {v}</li>"
    html += "</ul></details>"
    
    html += "<details open><summary><b>Functions</b></summary><ul>"
    for k, v in user_fns.items():
        html += f"<li><code>{k}{v}</code></li>"
    html += "</ul></details>"
    
    display(HTML(html))

# Auto-show after each cell (optional)
# get_ipython().events.register('post_run_cell', lambda r: show_user_items())
```

#### Provide smarter code completion and signature help

Why These Are Better Than Default IPython

Default IPython completion:
- No custom ranking (alphabetical or frequency-based)
- Includes all __dunder__ methods always
- Doesn't prioritize user code

These custom versions:
- Smart ranking: user code > builtins > everything else
- Filters __dunder__ unless explicitly requested
- Context-aware (knows about your namespace)
- Provides structured data for rich UI display

```python
InteractiveShell.ranked_complete(self:InteractiveShell, code, line_no=None, col_no=None)
```

Purpose: Provide intelligent, ranked code completions using Jedi, with custom ranking logic.

Parameters:
- code: The code string to complete
- line_no: Optional line number (1-indexed)
- col_no: Optional column number (1-indexed)

Returns: List of completion objects with these attributes:
- text - The completion text
- type - Type of completion (param, function, module, etc.)
- mod - Module where the item is defined
- rank - Numeric rank (lower = higher priority)

Ranking logic (lower is better):
- Rank 1: Parameters (function arguments being filled)
- Rank 2: Local variables/functions (from __main__)
- Rank 3: Module members (when completing module.something)
- Rank 4: Builtins (like print, len, etc.)
- Rank 5: Everything else (imported modules, third-party)
- Rank +0.1: Private items (starting with _) - slightly lower priority

Special handling:
- Filters out __dunder__ methods unless the user explicitly types __ (so they're not cluttering normal completion)
- Deprioritizes private _methods slightly but doesn't remove them

Use case: Provide better autocomplete suggestions in the solveit dialog environment, prioritizing user-defined items and parameters over stdlib/third-party items.

```python
InteractiveShell.sig_help(self:InteractiveShell, code, line_no=None, col_no=None)
```

Purpose: Get function signature information at the cursor position (like when you type func( and want to see the parameters).

Parameters:
- code	The code string
- line_no	Line number where cursor is (1-indexed)
- col_no	Column number where cursor is (1-indexed)

Returns: List of signature objects, each containing:
- label - Full signature description (e.g., "print(value, ..., sep=' ', end='\n')")
- typ - Type of callable (function, method, class, etc.)
- mod - Module name where it's defined
d- oc - Full docstring
- idx - Current parameter index (which parameter the cursor is on)
- params - List of parameter dicts with name and desc

How it works:
- Uses Jedi's Interpreter with the current namespace to get context-aware signatures
- Falls back to Script (static analysis) if Interpreter doesn't find anything
- Extracts detailed information about each signature
- Returns structured data about parameters and documentation

Use case: Power the signature help tooltip in the solveit editor, showing:
- What parameters a function takes
- Which parameter you're currently typing
- Documentation for each parameter
- Full docstring


```python
Inspector._get_info(self:Inspector, obj, oname='', formatter=None, info=None, detail_level=0, omit_sections=())
```

Purpose: Customizes the ?? (double question mark) output to display source code as formatted Markdown.

Note: This is patched onto Inspector, not InteractiveShell.

Parameters:
- obj: The object being inspected
- oname 	Object name (string)
- formatter: Optional formatter
- info: Pre-computed info dict
- detail_level: 0 = basic (?), 1 = detailed (??)
- omit_sections: Sections to skip

How it works:
- Calls the original _get_info method first (stored as _orig__get_info)
- If detail_level == 0 (single ?), returns original output unchanged
- If detail_level == 1 (double ??), creates enhanced Markdown output:
  - Source code in a Python fenced code block
  - File path in bold with backticks

Use case: Makes the ?? inspection output much more readable in environments that support Markdown rendering

#### Override IPython's default behavior to enhance completions

JupyterLab communicates with the kernel via the Jupyter messaging protocol:

```
┌─────────────────┐                    ┌─────────────────┐
│   JupyterLab    │  complete_request  │    IPython      │
│   Frontend      │ ─────────────────► │    Kernel       │
│                 │                    │                 │
│                 │ ◄───────────────── │                 │
│                 │  complete_reply    │                 │
└─────────────────┘                    └─────────────────┘
```

The relevant message types are:
- complete_request / complete_reply - for autocompletion
- inspect_request / inspect_reply - for signature/documentation help

**Patch do_complete on the Kernel**

The kernel's do_complete method handles complete_request messages. You can patch it:

```python
from ipykernel.ipkernel import IPythonKernel
from functools import wraps

# Store original
_original_do_complete = IPythonKernel.do_complete

@wraps(_original_do_complete)
def custom_do_complete(self, code, cursor_pos):
    """Enhanced completion with ranking."""
    # Get original reply
    reply = _original_do_complete(self, code, cursor_pos)
    
    if reply['status'] == 'ok':
        matches = reply['matches']
        
        # Custom ranking logic
        def rank(name):
            ...
        
        # Filter __dunder__ unless explicitly typing them
        text = code[:cursor_pos].split()[-1] if code[:cursor_pos].split() else ''
        if '__' not in text:
            matches = [m for m in matches if not m.startswith('__')]
        
        # Sort by rank
        reply['matches'] = sorted(matches, key=rank)
    
    return reply

# Apply patch
IPythonKernel.do_complete = custom_do_complete
```

**Patch do_inspect for Signature Help**

For enhanced signature/documentation help:

```python
from ipykernel.ipkernel import IPythonKernel
from jedi import Interpreter
from functools import wraps

_original_do_inspect = IPythonKernel.do_inspect

@wraps(_original_do_inspect)
def custom_do_inspect(self, code, cursor_pos, detail_level=0):
    """Enhanced inspect with Jedi signatures."""
    # Try Jedi first for better signature info
    try:
        ns = self.shell.user_ns
        
        # Get signatures at cursor
        ...
            
            return {
                'status': 'ok',
                'found': True,
                'data': {'text/markdown': text, 'text/plain': sig.docstring()},
                'metadata': {}
            }
    except Exception:
        pass
    
    # Fall back to original
    return _original_do_inspect(self, code, cursor_pos, detail_level)

IPythonKernel.do_inspect = custom_do_inspect
```

**Create an IPython Extension**

Package everything as a proper IPython extension:

```python
def load_ipython_extension(ip):
    """Called when extension is loaded via %load_ext"""
    from ipykernel.ipkernel import IPythonKernel
    from functools import wraps
    
    # Patch do_complete
    _orig = IPythonKernel.do_complete
    
    @wraps(_orig)
    def patched_do_complete(self, code, cursor_pos):
        ...
    
    IPythonKernel.do_complete = patched_do_complete
    print("Enhanced completions loaded!")

def unload_ipython_extension(ip):
    """Called when extension is unloaded"""
    # Restore original if needed
    pass
```

Usage:

```
%load_ext my_completer_extension
```

Or add to ipython_config.py:

```
c.InteractiveShellApp.extensions = ['my_completer_extension']
```

#### Programmatically inject variables into the user namespace

```python
InteractiveShell.xpush(self:InteractiveShell, interactive=False, **kw):
    "Like `push`, but with kwargs"
    self.push(kw, interactive=interactive)
```

Purpose: A convenience wrapper around InteractiveShell.push() that accepts keyword arguments instead of a dictionary.

Parameters:
- interactive: If True, the variables are treated as if typed interactively (affects display)
- **kw: Variables to inject as keyword arguments

Comparison:

```python
shell = get_ipython()

# Standard push() - requires a dictionary
shell.push({'x': 42, 'name': 'Alice'})

# xpush() - cleaner kwargs syntax
shell.xpush(x=42, name='Alice')
```

Use case: Cleaner API when programmatically injecting variables into the user namespace.

**What InteractiveShell.push() does**

Purpose: Programmatically add variables to the user's namespace - as if the user had typed them directly in the notebook.

The interactive Parameter controls how the variables are treated:
- interactive=True (default):
  - Variables treated as if user typed them
  - Subject to display hooks (e.g., last expression displays)
  - Triggers post_execute hooks
  - Can show output
- interactive=False:
  - Variables injected silently
  - No display output
  - Minimal side effects
  - Used for "behind the scenes" injection

**Use Cases in solveit**

1. Tool Results Injection

When the AI calls a tool, the result needs to be available to the user:

```python
# AI calls a tool: calculate(5, 3)
def calculate(a: int, b: int) -> int:
    return a + b

result = calculate(5, 3)  # Tool executes

# Inject result into user namespace
shell.xpush(tool_result=result, interactive=False)

# User can now access it:
print(tool_result)  # 8
```

Where: `claudette.toolloop` / `cosette.toolloop` / `lisette.core`

File & Function:
```python
# claudette/toolloop.py
@patch
def toolloop(self:Chat, pr, max_steps=10, ...):
    # After tool execution, results are automatically added to chat history
    # The tool result becomes part of the conversation context
```

Implementation: Tool results are typically kept in the chat history rather than pushed to namespace. However, if you wanted to inject them:

```python
# dialoghelper/core.py (hypothetical)
def execute_tool_and_inject(tool_name, args, ns):
    result = call_func(tool_name, args, ns)
    shell = get_ipython()
    shell.xpush(**{f'{tool_name}_result': result}, interactive=False)
    return result
```

Actual location: Not directly implemented in the libraries we explored - this would be custom solveit backend code.

2. Loading Context from External Sources

When importing from a gist or URL:

```python
# User runs: import_gist('abc123')
# Behind the scenes:
gist_code = fetch_gist('abc123')
exec(gist_code, globals_dict := {})

# Inject all functions/classes from gist
shell.xpush(interactive=False, **globals_dict)

# Now user has access to everything from the gist
```

Where: `dialoghelper.core`

File & Function:
```python
# dialoghelper/core.py
def import_gist(gist_id:str, mod_name:str='gist', run:bool=False)
```

Key line: `shell.user_ns[k] = v` - directly modifies namespace (equivalent to `push`)

3. AI-Generated Variables

When AI generates code that creates variables:

```python
# AI generates this code:
code = """
import pandas as pd
df = pd.DataFrame({'a': [1,2,3], 'b': [4,5,6]})
"""

# Execute and capture namespace
exec_namespace = {}
exec(code, exec_namespace)

# Push the dataframe to user
shell.xpush(df=exec_namespace['df'])

# User now has 'df' available
df.head()
```

Where: `execnb.shell` / `toolslm.shell`

File & Function:
```python
# toolslm/shell.py
def get_shell():
    "Get a `TerminalInteractiveShell` with minimal functionality"
    # Shell maintains its own namespace
    # Variables from executed code live in shell.user_ns
```

```python
# execnb/shell.py (from execnb library)
class CaptureShell:
    def run(self, code):
        # Executes code and captures outputs
        # Variables stay in shell's namespace
```

For injecting into main namespace: Custom solveit code would do:
```python
shell = get_ipython()
shell.xpush(**exec_namespace, interactive=False)
```

4. Sharing Variables Between Messages

When running code in one message and needing results in another:

```python
# Message 1 (code cell):
x = expensive_computation()

# Behind the scenes, solveit might:
shell.xpush(_last_result=x, interactive=False)

# Message 2 (prompt to AI):
# AI can reference _last_result
```

Where: `dialoghelper.core`

File & Function:
```python
# dialoghelper/core.py

def add_msg(content, placement='afterCurrent', msgid=None, msg_type='note', ...):
    "Add a message to dialog"
    # Messages can contain code that executes
    # Results are automatically in namespace

def run_msg(msgid, dname=None):
    "Queue a message for execution"
    # Executes code cell, results go to namespace automatically
```

Implementation: When code cells execute in solveit, they naturally share the same namespace. No explicit `push()` needed - it's the default behavior.

5. Pre-loading Helper Functions

When starting a dialog, inject helper utilities:

```python
# On dialog start
from dialoghelper import read_msg, add_msg, update_msg

shell.xpush(
    read_msg=read_msg,
    add_msg=add_msg,
    update_msg=update_msg,
    interactive=False
)

# Now available everywhere in the dialog
```

Where: `ipykernel_helper.core`

File & Function:
```python
# ipykernel_helper/core.py
def load_ipython_extension(ip):
    "Load extension and inject helper functions"
    from ipykernel_helper import transient, run_cmd
    ns = ip.user_ns
    ns['read_url'] = read_url
    ns['transient'] = transient
    ns['run_cmd'] = run_cmd
```

Key line: `ns['read_url'] = read_url` - directly modifies namespace dictionary

Also:
```python
# dialoghelper/core.py (hypothetical startup)
def initialize_dialog_helpers():
    shell = get_ipython()
    shell.xpush(
        read_msg=read_msg,
        add_msg=add_msg,
        update_msg=update_msg,
        find_msgs=find_msgs,
        interactive=False
    )
```

6. Restoring Session State

When loading a saved dialog:

```python
# Load saved state
saved_vars = load_dialog_state('dialog_123')
# saved_vars = {'x': 42, 'data': [...], 'model': <object>}

# Restore to namespace
shell.xpush(interactive=False, **saved_vars)

# User's variables are back
```

Where: Not explicitly implemented in the libraries we explored

Would be in: Custom solveit backend (not open source)

Hypothetical implementation:
```python
# solveit/session.py (hypothetical)

def restore_dialog_state(dialog_id):
    "Restore variables from saved dialog"
    import pickle
    
    # Load saved state
    with open(f'dialogs/{dialog_id}/state.pkl', 'rb') as f:
        saved_vars = pickle.load(f)
    
    # Restore to namespace
    shell = get_ipython()
    shell.xpush(interactive=False, **saved_vars)
    
    return saved_vars
```

**Key Insight**

Most of these behaviors don't explicitly call `push()` or `xpush()` - they use alternative approaches:

1. **Direct namespace modification**: `shell.user_ns['key'] = value`
2. **Natural code execution**: Code cells share namespace automatically
3. **Extension loading**: Inject at startup via `load_ipython_extension()`

The `xpush()` convenience wrapper is available but many implementations use the underlying mechanisms directly. The actual solveit backend (which orchestrates dialogs, AI interactions, and message execution) likely uses `xpush()` more extensively, but that code isn't in the open-source libraries we explored.

**Architecture: How solveit Uses push()**

```
┌─────────────────────────────────────────────────────────────────────┐
│                         USER ACTION                                 │
│   • Runs a tool                                                     │
│   • Imports a gist                                                  │
│   • AI generates code                                               │
│   • Loads dialog state                                              │
└────────────────────────────┬────────────────────────────────────────┘
                             │
                             ▼
┌─────────────────────────────────────────────────────────────────────┐
│                    SOLVEIT BACKEND                                  │
│   • Executes code in isolated namespace                             │
│   • Captures results                                                │
│   • Determines what to share with user                              │
└────────────────────────────┬────────────────────────────────────────┘
                             │
                             ▼
┌─────────────────────────────────────────────────────────────────────┐
│                    shell.xpush()                                    │
│   • Injects variables into user namespace                           │
│   • Makes results accessible in subsequent cells                    │
└────────────────────────────┬────────────────────────────────────────┘
                             │
                             ▼
┌─────────────────────────────────────────────────────────────────────┐
│                    USER NAMESPACE                                   │
│   Variables now available:                                          │
│   • print(tool_result)                                              │
│   • df.head()                                                       │
│   • my_imported_function()                                          │
└─────────────────────────────────────────────────────────────────────┘
```

**Example: Complete Tool Execution Flow**

```python
# 1. User shares a tool with AI
def fetch_data(url: str) -> dict:
    """Fetch JSON from URL"""
    import httpx
    return httpx.get(url).json()

# 2. AI decides to call it
# Behind the scenes in solveit:
tool_name = "fetch_data"
tool_args = {"url": "https://api.github.com/users/torvalds"}

# 3. Execute tool
from toolslm.funccall import call_func
result = call_func(tool_name, tool_args, namespace={'fetch_data': fetch_data})

# 4. Inject result into user namespace
shell = get_ipython()
shell.xpush(
    github_data=result,  # Make result accessible
    interactive=False     # Silent injection
)

# 5. User can now use it:
print(github_data['name'])  # 'Linus Torvalds'
```

**Why Not Just Execute Directly?**

You might wonder: why use `push()` instead of just executing code directly?

Problems:
- Harder to control scope
- Difficult to inject complex objects
- No clean separation between execution and namespace injection
- Can't easily inject from external sources

Benefits of using push:
- Clean API for namespace manipulation
- Works with any Python object (including unpicklable ones)
- Respects IPython's namespace management
- Can control interactive behavior
- Integrates with IPython's hooks and events

**Summary**

| Feature | Purpose in solveit |
|---------|-------------------|
| Tool results | Make AI tool outputs available to user |
| Gist imports | Inject functions from GitHub gists |
| AI code execution | Share variables from AI-generated code |
| Session restoration | Reload saved dialog state |
| Helper injection | Pre-load utility functions |
| Context sharing | Pass data between messages/cells |

| Use Case | Library | File | Function/Method |
|----------|---------|------|-----------------|
| 1. Tool Results | (custom solveit) | N/A | Not in open source libs |
| 2. Gist Imports | `dialoghelper` | `core.py` | `import_gist()` |
| 3. AI-Generated Vars | `execnb` / `toolslm` | `shell.py` | `CaptureShell.run()` / `get_shell()` |
| 4. Message Variables | `dialoghelper` | `core.py` | `run_msg()`, `add_msg()` |
| 5. Helper Pre-loading | `ipykernel_helper` | `core.py` | `load_ipython_extension()` |
| 6. Session Restoration | (custom solveit) | N/A | Not in open source libs |
    
Key insight: `push()` / `xpush()` is the bridge between solveit's backend execution and the user's interactive namespace. It's how results from AI actions become available for the user to work with.

#### Send commands and display data to the Javascript frontend

```python
InteractiveShell.run_cmd(cmd, data='', meta=None, update=False, **kw)
```

Purpose: a convenience wrapper that sends a command to the frontend via the transient mechanism.

Internally using: `InteractiveShell.transient()`.

In Jupyter, when you output something, it gets sent to the frontend via a display_data or execute_result message. These messages have three main parts:

```json
{
    "data": {"text/plain": "Hello", "text/html": "<b>Hello</b>"},  # The content
    "metadata": {},  # Extra info about the content
    "transient": {}  # Data that should NOT be persisted in the notebook
}
```

Sends display data to the frontend where the actual payload is in the transient field, not the main data field.

The key insight: transient data is displayed but not saved when the notebook is saved. It's ephemeral.

Why use transient?
- Not saved to notebook - Commands and temporary UI updates don't clutter the saved .ipynb file
- Custom frontend communication - The solveit frontend watches for specific transient keys
- Ephemeral state - Progress indicators, status updates, commands that shouldn't persist

Example usage: `run_cmd("scroll_to", msg_id="abc123")`

```python
InteractiveShell.publish(self:InteractiveShell, data='', subtype='plain', mimetype='text', meta=None, update=False, **kw)

InteractiveShell.transient(data='', subtype='plain', mimetype='text', meta=None, update=False, **kw)
```

Purpose: A flexible method to publish display data to the frontend, with support for transient data.

Parameters:
- data: Content to display (string, DisplayObject, or dict)
- subtype: MIME subtype (default: 'plain')
- mimetype: MIME type (default: 'text') → combined as text/plain
- meta: Metadata dictionary
- update: If True, updates a previous display with same -display_id
- **kw: Extra kwargs go into the transient field

How it works:
- If data is a DisplayObject (like HTML, Markdown), it formats it properly
- If data is not a dict/mapping, it wraps it as {mimetype/subtype: data}
- Publishes via display_pub.publish() with transient data in **kw

Examples:

```python
shell = get_ipython()

# Publish plain text
shell.publish("Hello, world!")

# Publish HTML
shell.publish("<b>Bold text</b>", subtype='html')

# Publish with transient data (for frontend commands)
shell.publish("status", cmd="update_status", id="123")

# Publish a DisplayObject
from IPython.display import HTML
shell.publish(HTML("<h1>Title</h1>"))

# Update an existing display
shell.publish("Updated content", update=True, display_id="my_display")
```

publish() is the lower-level method that transient() essentially wraps.

### Web access

I have access to two main tools:

1. Web Search - I can search for current information, recent events, technical documentation, news, and anything where up-to-date data would be helpful. I use this when:
- You need recent information (after my March 2025 knowledge cutoff)
- You're looking for specific facts or current conditions
- Real-time data is important (weather, news, stock prices, etc.)

2. Read URL - I can fetch and read content from specific web pages you provide, which is useful for analyzing articles, documentation, or other online content.

I use these tools strategically - I don't search for things I already know well (like general programming concepts, historical facts, or established knowledge), but I will search when current or specific information would genuinely help answer your question.

#### Model inference service

The web_search function isn't defined in your Python environment - it's a tool that's available to me (the AI assistant) but not directly accessible to you in your Python code.

When I use web_search, I'm calling it through my own tool-calling mechanism, not through your Python interpreter. It's part of my capabilities, separate from the Python environment you're working in.

```python
web_search(query: str)
```

Parameters
- Takes a search query string and returns web search results

#### ipykernel_helper

Full-featured URL reader that can extract sections, convert to markdown, handle math (KaTeX/MathJax), absolutify image URLs, and optionally tag images for AI processing

```python
read_url(
    url: str,
    as_md: bool = True,
    extract_section: bool = True,
    selector: str = None
)
```

Parameters
- url: The URL to read (required)
- as_md: Whether to convert HTML to Markdown (default: True)
- extract_section: If URL has an anchor, return only that section (default: True)
- selector: CSS selector to extract specific sections using BeautifulSoup.select syntax (optional)

Uses internally
- scrape_url() - Fetch URL content using cloudscraper (handles anti-bot protections)
- get_md() - Convert HTML to clean markdown


### Code execution

In [ ]:
#| export
# import ...